In [1]:
# import os
# os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

import StellBlanket.SBGeom as SBGeom
from functools import partial
import jax_sbgeom.coils as jsc

from tests.coils.test_coils import _check_single_vectorized_internal
import pyvista as pv
from dataclasses import dataclass


vmec_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"]
vmec_file = vmec_files[2]

coil_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5"]

Triangle Elements not compiled


In [2]:
def _get_discrete_coils(coil_file, coil_index=0):
    sbgeom_coils = SBGeom.Coils.Discrete_Coil_Set_From_HDF5(coil_file)
    print(sbgeom_coils.Number_of_Coils())
    coil_0 = sbgeom_coils[coil_index]
    coil_0_jsc = jsc.DiscreteCoil.from_positions(coil_0.Get_Vertices())
    return coil_0_jsc, coil_0

c_jax, c_sbgeom = _get_discrete_coils(coil_files[2])

40


In [3]:
class bb:
    param : str
    def __init__(self, param):
        self.param = param

def _get_all_discrete_coils(request):    
    coilset_sbgeom = SBGeom.Coils.Discrete_Coil_Set_From_HDF5(request.param)    
    coilset_jaxsbgeom = jsb.coils.CoilSet([jsb.coils.DiscreteCoil.from_positions(coilset_sbgeom[i].Get_Vertices()) for i in range(coilset_sbgeom.Number_of_Coils())])
    
    return coilset_jaxsbgeom, coilset_sbgeom


jsbc, sbgc = _get_all_discrete_coils(bb(coil_files[1]))

In [4]:
s = jnp.linspace(0, 1.0, 1000, endpoint =False)
filament = jsbc[0].position(s)
radial_vec = jsc.discrete_coil._discrete_coil_compute_rmf(jsbc[0], 1000)


In [5]:
ns_reinterp = 100

s_reinterp = jnp.linspace(0, 1.0, ns_reinterp, endpoint=False)
s_reinterp = 0.1

radial_vec_new = jsc.discrete_coil._discrete_coil_rmf(jsbc[0], s_reinterp, 1000)

In [6]:
filament_new = jsbc[0].position(s_reinterp)

In [7]:
def ref(s):
    return jsc.discrete_coil._discrete_coil_rmf(jsbc[0], s, 1000)

In [8]:
ns = 3
s_new = jnp.linspace(0, 1.0, ns, endpoint=False)
finite_sizes =jsc.discrete_coil._discrete_coil_finite_size(jsbc[0], s_new, 0.3,0.3, method="rmf", number_of_rmf_samples=ns)

[[-0.13092573  0.06594186  0.26174438]
 [-0.220478   -0.19395058 -0.06142169]]


In [9]:
f3 = jnp.moveaxis(finite_sizes, 1, 0)

In [10]:
finite_sizes =jsc.discrete_coil._discrete_coil_finite_size(jsbc[0], s_new, 0.3,0.3, method="rmf", number_of_rmf_samples=ns)

[[-0.13092573  0.06594186  0.26174438]
 [-0.220478   -0.19395058 -0.06142169]]


In [11]:
ns = 5
s_new = jnp.linspace(0, 1.0, ns, endpoint=False)
finite_sizes =jsc.discrete_coil._discrete_coil_finite_size(jsbc[0], s_new, 0.3,0.3, method="rmf", number_of_rmf_samples=ns)
finite_sizes2 = sbgc[0].Finite_Size_Lines_RMF(0.3, 0.3, ns)
f3 = jnp.moveaxis(finite_sizes, 1, 0)
print(f3)
print(finite_sizes2)

[[-0.13092573  0.06594186  0.26174438]
 [-0.220478   -0.19395058 -0.06142169]]
POS 0
 POS:
20.1634
17.3145
 0.7402
DISPR
-0.130926
0.0659419
 0.261744
DISPPHI
  -0.220478
 -0.193951
-0.0614217
POS 1
 POS:
18.6291
14.0345
 2.9819
DISPR
 0.0441167
  0.290216
-0.0618734
DISPPHI
  -0.084879
-0.0475981
 -0.283778
POS 2
 POS:
12.9731
13.1064
 2.4738
DISPR
-0.148627
  0.08976
-0.244649
DISPPHI
  0.221337
-0.105178
-0.173054
POS 3
 POS:
13.0612
10.4064
-3.7326
DISPR
0.00068976
  0.252449
  0.162077
DISPPHI
  0.229928
-0.104552
  0.16187
POS 4
 POS:
18.3134
14.6367
-3.2566
DISPR
 -0.173819
-0.0302045
  0.242641
DISPPHI
    0.23649
-0.0964059
  0.157411
[[[19.81194874 17.18648692  0.9405227 ]
  [18.58833566 14.2770687   2.63624857]
  [13.0458056  13.09097484  2.05609718]
  [13.29183387 10.55434169 -3.408653  ]
  [18.37607661 14.5101025  -2.85654738]]

 [[20.0738002  17.05460319  0.41703393]
  [18.5001023  13.69663644  2.75999536]
  [13.34306022 12.9114548   2.54539488]
  [13.29045435 10.04944308

In [12]:

plotter = pv.Plotter()
pline = pv.Spline(filament)
plotter.add_mesh(pline, color='red', line_width=2)

plotter.add_arrows(onp.array(filament), onp.array(radial_vec), mag=0.2, color='blue')
plotter.add_arrows(onp.array(filament_new), onp.array(radial_vec_new), mag=0.2, color='red')

plotter.show()

Widget(value='<iframe src="http://localhost:43183/index.html?ui=P_0x7ca7e77246e0_0&reconnect=auto" class="pyvi…

In [13]:
ns = 7000
coil = c_sbgeom
jsb_coils = [c_jax]
s_new = jnp.linspace(0.0, 1.0, ns, endpoint=False)
centroid_0 = coil.Finite_Size_Lines_Centroid(0.3, 0.5, ns) 

centroid_1 = c_jax.finite_size(s_new, 0.5, 0.3, "centroid")
centroid_2 = jnp.moveaxis(centroid_1, 1,0).reshape(-1,3)

jnp.allclose(centroid_0, centroid_2, atol=1e-7)

[[ 0.49424112  0.02022878 -0.07291442]
 [ 0.0083912   0.26944858  0.13163225]]


Array(True, dtype=bool)